###  Data Loading & Preprocessing

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery as bq

service_account_path = "/home/yusuf/DataScience/dream_games/ybektas20.json" 
client = bq.Client.from_service_account_json(service_account_path)

query = """
SELECT
  table_name,
  column_name,
  data_type,
  is_nullable,
  ordinal_position
FROM `casedreamgames.case_db.INFORMATION_SCHEMA.COLUMNS`
ORDER BY table_name, ordinal_position;
"""
db_info = client.query(query).result().to_dataframe()
db_info.groupby(["table_name"]).apply(lambda x: print(x), include_groups=False)

/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


  column_name data_type is_nullable  ordinal_position
0        date      DATE         YES                 1
1     network    STRING         YES                 2
2     country    STRING         YES                 3
3    platform    STRING         YES                 4
4        cost   FLOAT64         YES                 5
  column_name  data_type is_nullable  ordinal_position
5  event_time  TIMESTAMP         YES                 1
6     user_id     STRING         YES                 2
7    platform     STRING         YES                 3
8     network     STRING         YES                 4
9     country     STRING         YES                 5
   column_name  data_type is_nullable  ordinal_position
10  event_time  TIMESTAMP         YES                 1
11     user_id     STRING         YES                 2
12    platform     STRING         YES                 3
13       level      INT64         YES                 4
14      status     STRING         YES                 5
15  time_s

""


In [2]:
# Define queries for each Q1 table
q1_tables_queries = {
    "q1_table_install": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(network) AS non_null_network,
          COUNT(country) AS non_null_country,
          COUNT(event_time) AS non_null_event_time
        FROM `casedreamgames.case_db.q1_table_install`;
    """,
    "q1_table_level_end": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(level) AS non_null_level,
          COUNT(status) AS non_null_status,
          COUNT(time_spent) AS non_null_time_spent,
          COUNT(moves_made) AS non_null_moves_made,
          COUNT(moves_left) AS non_null_moves_left,
          COUNT(event_time) AS non_null_event_time
        FROM `casedreamgames.case_db.q1_table_level_end`;
    """,
    "q1_table_session": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(coin_status) AS non_null_coin_status,
          COUNT(time_spent) AS non_null_time_spent,
          COUNT(level) AS non_null_level,
          COUNT(event_time) AS non_null_event_time
        FROM `casedreamgames.case_db.q1_table_session`;
    """,
    "q1_table_revenue": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(user_id) AS non_null_user_id,
          COUNT(platform) AS non_null_platform,
          COUNT(package_type) AS non_null_package_type,
          COUNT(revenue) AS non_null_revenue,
          COUNT(event_time) AS non_null_event_time
        FROM `casedreamgames.case_db.q1_table_revenue`;
    """,
    "q1_table_cost": """
        SELECT
          COUNT(*) AS total_rows,
          COUNT(date) AS non_null_date,
          COUNT(network) AS non_null_network,
          COUNT(platform) AS non_null_platform,
          COUNT(country) AS non_null_country,
          COUNT(cost) AS non_null_cost
        FROM `casedreamgames.case_db.q1_table_cost`;
    """
}

# Iterate through each query, run it, and print the shape and null counts per column
for table_name, query in q1_tables_queries.items():
    print(f"Results for {table_name}:")
    
    # Run the query; this assumes that 'client' is your BigQuery client (or use your analytics object)
    df = client.query(query).result().to_dataframe()
    
    # Extract total row count
    total_rows = df.loc[0, "total_rows"]
    print(f"Shape: ({total_rows} rows)")
    
    # For each column (ignoring the total_rows column), compute and print null counts
    for col in df.columns:
        if col != "total_rows":
            non_null_count = df.loc[0, col]
            null_count = total_rows - non_null_count
            # Extract original column name by removing "non_null_" prefix
            orig_col = col.replace("non_null_", "")
            print(f"Column '{orig_col}': non-null = {non_null_count}, null = {null_count}")
    
    print("\n" + "-"*50 + "\n")


Results for q1_table_install:


/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Shape: (217415 rows)
Column 'user_id': non-null = 217415, null = 0
Column 'platform': non-null = 217415, null = 0
Column 'network': non-null = 217415, null = 0
Column 'country': non-null = 217415, null = 0
Column 'event_time': non-null = 217415, null = 0

--------------------------------------------------

Results for q1_table_level_end:
Shape: (72044374 rows)
Column 'user_id': non-null = 72044374, null = 0
Column 'platform': non-null = 72044373, null = 1
Column 'level': non-null = 72044374, null = 0
Column 'status': non-null = 72044374, null = 0
Column 'time_spent': non-null = 72044370, null = 4
Column 'moves_made': non-null = 72044373, null = 1
Column 'moves_left': non-null = 72044370, null = 4
Column 'event_time': non-null = 72044374, null = 0

--------------------------------------------------

Results for q1_table_session:
Shape: (297358858 rows)
Column 'user_id': non-null = 297358858, null = 0
Column 'platform': non-null = 297358850, null = 8
Column 'coin_status': non-null = 2973

### User Acquisition & Daily Active Users (DAU)

In [3]:
query = """
WITH session_data AS (
    SELECT 
      DATE(event_time) AS date, 
      user_id
    FROM `casedreamgames.case_db.q1_table_session`
)
SELECT
    s.date,
    COALESCE(i.country, 'unknown') AS country,
    COALESCE(i.platform, 'unknown') AS platform,
    COALESCE(i.network, 'unknown') AS network,
    COUNT(DISTINCT s.user_id) AS dau
FROM session_data s
LEFT JOIN `casedreamgames.case_db.q1_table_install` i
    ON s.user_id = i.user_id
GROUP BY s.date, country, platform, network
ORDER BY s.date, country, platform, network;     
"""
dau = client.query(query).result().to_dataframe()
dau.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1814 entries, 0 to 1813
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      1814 non-null   dbdate
 1   country   1814 non-null   object
 2   platform  1814 non-null   object
 3   network   1814 non-null   object
 4   dau       1814 non-null   Int64 
dtypes: Int64(1), dbdate(1), object(3)
memory usage: 72.8+ KB


In [7]:
import pandas as pd
import plotly.express as px


# Ensure the date column is in datetime format
dau['date'] = pd.to_datetime(dau['date'])

fig_total = px.line(
    dau.groupby("date", as_index=False).agg({"dau": "sum"}),
    x="date",
    y="dau",
    title="Daily Active Users",
    markers=True
)
fig_total.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="DAU")
fig_total.show()

fig_country = px.line(
    dau.groupby(["date", "country"], as_index=False).agg({"dau": "sum"}),
    x="date",
    y="dau",
    color="country",
    markers=True,
    title="Daily Active Users by Country",
)
fig_country.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="DAU")
fig_country.show()

fig_platform = px.line(
    dau.groupby(["date", "platform"], as_index=False).agg({"dau": "sum"}),
    x="date",
    y="dau",
    color="platform",
    markers=True,
    title="Daily Active Users by Platform"
)
fig_platform.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="DAU")
fig_platform.show()

fig_network = px.line(
    dau.groupby(["date", "network"], as_index=False).agg({"dau": "sum"}),
    x="date",
    y="dau",
    color="network",
    markers=True,
    title="Daily Active Users by Network"
)
fig_network.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="DAU")
fig_network.show()


### User Engagement and Retention

In [8]:
query ="""
WITH installs AS (
  SELECT 
    user_id, 
    DATE(event_time) AS install_date,
    country,
    platform,
    network
  FROM `casedreamgames.case_db.q1_table_install`
),
sessions AS (
  SELECT 
    user_id, 
    DATE(event_time) AS session_date
  FROM `casedreamgames.case_db.q1_table_session`
)
SELECT
  DATE_ADD(i.install_date, INTERVAL 1 DAY) AS retention_date,
  i.country,
  i.platform,
  i.network,
  COUNT(DISTINCT i.user_id) AS installs_previous_day,
  COUNT(DISTINCT s.user_id) AS retained_today,
  SAFE_DIVIDE(COUNT(DISTINCT s.user_id), COUNT(DISTINCT i.user_id)) AS retention_rate
FROM installs i
LEFT JOIN sessions s
  ON i.user_id = s.user_id
  AND s.session_date = DATE_ADD(i.install_date, INTERVAL 1 DAY)
GROUP BY retention_date, i.country, i.platform, i.network
ORDER BY retention_date, i.country, i.platform, i.network;
"""
retention = client.query(query).result().to_dataframe()
retention.info()

/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926 entries, 0 to 925
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   retention_date         926 non-null    dbdate 
 1   country                926 non-null    object 
 2   platform               926 non-null    object 
 3   network                926 non-null    object 
 4   installs_previous_day  926 non-null    Int64  
 5   retained_today         926 non-null    Int64  
 6   retention_rate         926 non-null    float64
dtypes: Int64(2), dbdate(1), float64(1), object(3)
memory usage: 52.6+ KB


In [12]:
### plot it by total, country, platform and network
retention_total = retention.groupby(["retention_date"], as_index=False).agg({"retained_today": "sum", "installs_previous_day": "sum"})
retention_total["retention_rate"] = retention_total["retained_today"] / retention_total["installs_previous_day"]

fig_total = px.line(
    retention_total.iloc[1:-1],
    x="retention_date",
    y="retention_rate",
    title="Retention Rate",
    markers=True
)
fig_total.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Retention Rate")
fig_total.show()


retention_by_country = retention.groupby(["retention_date", "country"], as_index=False).agg({"retained_today": "sum", "installs_previous_day": "sum"})
retention_by_country["retention_rate"] = retention_by_country["retained_today"] / retention_by_country["installs_previous_day"]
fig_country = px.line(
    retention_by_country.iloc[1:-1],
    x="retention_date",
    y="retention_rate",
    color="country",
    title="Retention Rate by Country",
    markers=True
)
fig_country.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Retention Rate")
fig_country.show()


retention_by_platform = retention.groupby(["retention_date", "platform"], as_index=False).agg({"retained_today": "sum", "installs_previous_day": "sum"})
retention_by_platform["retention_rate"] = retention_by_platform["retained_today"] / retention_by_platform["installs_previous_day"]
fig_platform = px.line(
    retention_by_platform.iloc[1:-1],
    x="retention_date",
    y="retention_rate",
    color="platform",
    title="Retention Rate by Platform",
    markers=True
)

fig_platform.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Retention Rate")
fig_platform.show()




### Game Play 

In [15]:
query = """
WITH user_hourly AS (
  SELECT
    DATE(s.event_time) AS date,
    EXTRACT(DAYOFWEEK FROM s.event_time) AS day_of_week,
    EXTRACT(HOUR FROM s.event_time) AS hour_of_day,
    i.country,
    i.platform,
    i.network,
    s.user_id,
    SUM(s.time_spent) AS total_time_spent
  FROM `casedreamgames.case_db.q1_table_session` s
  JOIN `casedreamgames.case_db.q1_table_install` i
    ON s.user_id = i.user_id
  GROUP BY 
    date,
    day_of_week,
    hour_of_day,
    i.country,
    i.platform,
    i.network,
    s.user_id
)
SELECT
  date,
  day_of_week,
  hour_of_day,
  country,
  platform,
  network,
  SUM(total_time_spent) AS total_time_spent,
  COUNT(DISTINCT user_id) AS user_count
FROM user_hourly
GROUP BY 
  date,
  day_of_week,
  hour_of_day,
  country,
  platform,
  network
ORDER BY 
  date,
  day_of_week,
  hour_of_day,
  country,
  platform,
  network;

"""

time_spent = client.query(query).result().to_dataframe()
time_spent.info()

/home/yusuf/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1820: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34049 entries, 0 to 34048
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              34049 non-null  dbdate
 1   day_of_week       34049 non-null  Int64 
 2   hour_of_day       34049 non-null  Int64 
 3   country           34049 non-null  object
 4   platform          34049 non-null  object
 5   network           34049 non-null  object
 6   total_time_spent  34049 non-null  Int64 
 7   user_count        34049 non-null  Int64 
dtypes: Int64(4), dbdate(1), object(3)
memory usage: 2.2+ MB


In [23]:
### average time spent by user per day per country, platform and network
time_spent_by_date = time_spent.groupby(["date"], as_index=False).agg({"total_time_spent": "sum", "user_count": "sum"})
time_spent_by_date['avg_time_spent'] = time_spent_by_date['total_time_spent'] / time_spent_by_date['user_count']

fig_total = px.line(
    time_spent_by_date,
    x="date",
    y="avg_time_spent",
    title="Average Time Spent by User per Day",
    markers=True
)
fig_total.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Average Time Spent")
fig_total.show()

time_spent_by_date_by_country = time_spent.groupby(["date", "country"], as_index=False).agg({"total_time_spent": "sum", "user_count": "sum"})
time_spent_by_date_by_country['avg_time_spent'] = time_spent_by_date_by_country['total_time_spent'] / time_spent_by_date_by_country['user_count']

fig_country = px.line(
    time_spent_by_date_by_country,
    x="date",
    y="avg_time_spent",
    color="country",
    title="Average Time Spent by User per Day by Country",
    markers=True
)
fig_country.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Average Time Spent")
fig_country.show()

time_spent_by_date_by_platform = time_spent.groupby(["date", "platform"], as_index=False).agg({"total_time_spent": "sum", "user_count": "sum"})
time_spent_by_date_by_platform['avg_time_spent'] = time_spent_by_date_by_platform['total_time_spent'] / time_spent_by_date_by_platform['user_count']

fig_platform = px.line(
    time_spent_by_date_by_platform,
    x="date",
    y="avg_time_spent",
    color="platform",
    title="Average Time Spent by User per Day by Platform",
    markers=True
)

fig_platform.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Average Time Spent")
fig_platform.show()

time_spent_by_date_by_network = time_spent.groupby(["date", "network"], as_index=False).agg({"total_time_spent": "sum", "user_count": "sum"})
time_spent_by_date_by_network['avg_time_spent'] = time_spent_by_date_by_network['total_time_spent'] / time_spent_by_date_by_network['user_count']

fig_network = px.line(
    time_spent_by_date_by_network,
    x="date",
    y="avg_time_spent",
    color="network",
    title="Average Time Spent by User per Day by Network",
    markers=True
)
fig_network.update_layout(template="plotly_white", xaxis_title="Date", yaxis_title="Average Time Spent")
fig_network.show()


In [ ]:
### average time spent by user per dayofweek per country, platform and network
time_spent_by_dow = time_spent.groupby(["day_of_week"], as_index=False).agg({"total_time_spent": "sum", "user_count": "sum"})
time_spent_by_dow['avg_time_spent'] = time_spent_by_dow['total_time_spent'] / time_spent_by_dow['user_count']
 